In [49]:
import os
os.chdir("D:/Courses/CE 7090 -Statistical and Econometric Methods in Civil Engineering II/HomeWork/HW-1")

In [50]:
import pandas as pd
import numpy as np
from statsmodels.base.model import GenericLikelihoodModel
import statsmodels.api as sm
from scipy.stats import norm  # Use scipy for normal distribution functions
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [51]:
# Load the dataset
file_path = "D:/Courses/CE 7090 -Statistical and Econometric Methods in Civil Engineering II/HomeWork/HW-1/Data_HW1_Tobit Analysis.txt"
column_names = [f"x{i+1}" for i in range(83)]  # Adjust column names as needed
data = pd.read_csv(file_path, delim_whitespace=True, header=None, names=column_names)

In [66]:
# Preprocessing
dependent_var = "x83"  # Dependent variable (accident rate)
# independent_vars =  [ "x26", "x27", "x28",   "x78"]  # Candidate variables
independent_vars =  [ "x6", "x9", "x10", "x14" , "x20", "x26", "x27"]

In [67]:
# List of columns to include
columns_to_include =  independent_vars  + [dependent_var]

# Create a new DataFrame with the selected columns
df= data[columns_to_include]


In [68]:
df

,x6,x9,x10,x14,x20,x26,x27,x83
0,28.0,10,62.8,0.050000,97.000,0,2,46.702065
1,24.0,10,64.4,0.050000,98.000,0,2,0.000000
2,25.0,10,69.8,0.048000,99.000,0,2,18.072971
3,26.0,10,77.8,0.052000,100.000,0,2,39.982950
4,26.0,10,74.4,0.049333,98.000,0,2,35.910281
...,...,...,...,...,...,...,...,...
333,42.9,6,71.2,0.178000,92.500,0,3,0.000000
334,30.0,6,71.2,0.178000,92.500,0,3,0.000000
335,41.4,6,71.2,0.178000,92.500,0,3,0.000000
336,41.4,7,75.0,0.130500,99.375,0,2,40.572678


In [70]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
x6,338.0,30.513905,6.674351,15.900,25.000,29.350000,36.000000,48.20000
x9,338.0,10.428994,5.731581,4.000,7.000,9.000000,12.000000,29.00000
x10,338.0,82.103254,26.533307,43.000,63.800,75.000000,97.000000,176.80000
x14,338.0,0.130852,0.060611,0.036,0.088,0.119256,0.178000,0.37600
x20,338.0,98.403557,2.597181,86.500,97.000,100.000000,100.000000,100.00000
x26,338.0,0.307692,0.861301,0.000,0.000,0.000000,0.000000,8.00000
x27,338.0,2.088757,0.284815,2.000,2.000,2.000000,2.000000,3.00000
x83,338.0,41.291988,47.429400,0.000,0.000,31.712859,63.654493,350.40208


In [56]:
# Drop rows with missing values
df = df.dropna()

# Alternatively, fill missing values with mean or median (if applicable)
# df = df.fillna(df.median())


In [57]:
df

,x26,x27,x28,x78,x83
0,0,2,1,0,46.702065
1,0,2,1,0,0.000000
2,0,2,1,0,18.072971
3,0,2,1,0,39.982950
4,0,2,1,0,35.910281
...,...,...,...,...,...
333,0,3,0,0,0.000000
334,0,3,0,0,0.000000
335,0,3,0,1,0.000000
336,0,2,1,1,40.572678


In [58]:
# Censor the dependent variable at 0 (left-censoring)
df["x83"] = df["x83"].clip(lower=0)


In [59]:
from sklearn.preprocessing import StandardScaler

# Step 1: Ensure categorical variables have correct categories
categorical_vars = ["x28", "x78"]
numerical_vars = [var for var in independent_vars if var not in categorical_vars]

# Ensure the categorical variables have the correct categories (e.g., 2 categories for each)
for cat_var in categorical_vars:
    df[cat_var] = df[cat_var].astype("category")  # Convert to categorical
    # Restrict to exactly 2 categories
    df[cat_var] = df[cat_var].cat.set_categories([0, 1])

# Step 2: One-hot encode categorical variables with proper prefixes
df_encoded = pd.get_dummies(
    df,
    columns=categorical_vars,
    prefix=categorical_vars,  # Use variable names as prefixes
    drop_first=True  # Drop the first category to avoid multicollinearity
)

# Step 3: Standardize numerical variables
scaler = StandardScaler()
df_encoded[numerical_vars] = scaler.fit_transform(df_encoded[numerical_vars])





In [60]:
df_encoded

,x26,x27,x83,x28_1,x78_1
0,-0.357771,-0.312094,46.702065,1,0
1,-0.357771,-0.312094,0.000000,1,0
2,-0.357771,-0.312094,18.072971,1,0
3,-0.357771,-0.312094,39.982950,1,0
4,-0.357771,-0.312094,35.910281,1,0
...,...,...,...,...,...
333,-0.357771,3.204164,0.000000,0,0
334,-0.357771,3.204164,0.000000,0,0
335,-0.357771,3.204164,0.000000,0,1
336,-0.357771,-0.312094,40.572678,1,1


In [61]:
y = df_encoded[dependent_var]
X = df_encoded.drop(columns=[dependent_var])

In [62]:
X

,x26,x27,x28_1,x78_1
0,-0.357771,-0.312094,1,0
1,-0.357771,-0.312094,1,0
2,-0.357771,-0.312094,1,0
3,-0.357771,-0.312094,1,0
4,-0.357771,-0.312094,1,0
...,...,...,...,...
333,-0.357771,3.204164,0,0
334,-0.357771,3.204164,0,0
335,-0.357771,3.204164,0,1
336,-0.357771,-0.312094,1,1


In [63]:
from sklearn.preprocessing import StandardScaler
from statsmodels.base.model import GenericLikelihoodModel
from scipy.stats import norm
import statsmodels.api as sm
import numpy as np

class TobitModel(GenericLikelihoodModel):
    def __init__(self, endog, exog, left=None, right=None, **kwargs):
        super(TobitModel, self).__init__(endog, exog, **kwargs)
        self.left = left
        self.right = right

    def loglike(self, params):
        beta = params[:-1]
        sigma = params[-1]
        if sigma <= 0:
            return -np.inf

        Xb = np.dot(self.exog, beta)
        y = self.endog
        z = (y - Xb) / sigma
        ll = np.where(y > self.left, norm.logpdf(z) - np.log(sigma), norm.logcdf(z))
        return np.sum(ll)

# Standardize variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Add constant to the independent variables
X_scaled = sm.add_constant(X_scaled)

# Fit the Tobit model
model = TobitModel(y, X_scaled, left=0)

# Generate correct starting parameters
start_params = np.append(np.zeros(X_scaled.shape[1]), 1)  # One for each variable + sigma

# Fit the model using BFGS optimization
result = model.fit(start_params=start_params, method='bfgs', disp=True)

# Display results
print(result.summary())


Optimization terminated successfully.
         Current function value: 3.990533
         Iterations: 98
         Function evaluations: 103
         Gradient evaluations: 103
                              TobitModel Results                              
Dep. Variable:                    x83   Log-Likelihood:                -1348.8
Model:                     TobitModel   AIC:                             2710.
Method:            Maximum Likelihood   BIC:                             2733.
Date:                Tue, 28 Jan 2025                                         
Time:                        14:19:13                                         
No. Observations:                 338                                         
Df Residuals:                     333                                         
Df Model:                           4                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------

C:\Users\hq8312\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:2748: UserWarning: df_model + k_constant + k_extra differs from k_params
  warnings.warn("df_model + k_constant + k_extra "
C:\Users\hq8312\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:2752: UserWarning: df_resid differs from nobs - k_params
  warnings.warn("df_resid differs from nobs - k_params")


In [64]:
# Log-likelihood of the fitted model
LL_beta = result.llf

# Step 3: Fit the null model (only constant term)
X_null = sm.add_constant(np.ones_like(y))  # Null model (only constant)
null_model = TobitModel(y, X_null, left=0)
result_null = null_model.fit(start_params=np.array([0, 1]), method="bfgs", disp=False)

# Log-likelihood of the null model
LL_0 = result_null.llf

# Step 4: Calculate Maddala's R2
N = len(y)  # Number of observations
LRT = 2 * (LL_beta - LL_0)  # Likelihood Ratio Test statistic
Maddala_R2 = 1 - np.exp(-LRT / N)

# Output results
print(f"Log-Likelihood (Fitted Model): {LL_beta:.4f}")
print(f"Log-Likelihood (Null Model): {LL_0:.4f}")
print(f"Likelihood Ratio Test (LRT): {LRT:.4f}")
print(f"Maddala's R2: {Maddala_R2:.4f}")

Log-Likelihood (Fitted Model): -1348.8001
Log-Likelihood (Null Model): -1384.9738
Likelihood Ratio Test (LRT): 72.3473
Maddala's R2: 0.1927


C:\Users\hq8312\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:2748: UserWarning: df_model + k_constant + k_extra differs from k_params
  warnings.warn("df_model + k_constant + k_extra "
C:\Users\hq8312\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:2752: UserWarning: df_resid differs from nobs - k_params
  warnings.warn("df_resid differs from nobs - k_params")


In [65]:
from scipy.stats import chi2

# Given LRT value
LRT = 72.3473

# Degrees of freedom (number of independent variables)
df = 4

# Calculate critical value for alpha = 0.05 (95% confidence level)
alpha = 0.05
critical_value = chi2.ppf(1 - alpha, df)

# Determine significance
if LRT > critical_value:
    result = "The fitted model is significantly better than the null model."
else:
    result = "The fitted model is NOT significantly better than the null model."

# Print results
print(f"LRT: {LRT:.4f}")
print(f"Degrees of Freedom: {df}")
print(f"Chi-Square Critical Value (alpha=0.05): {critical_value:.4f}")
print(f"Result: {result}")


LRT: 72.3473
Degrees of Freedom: 4
Chi-Square Critical Value (alpha=0.05): 9.4877
Result: The fitted model is significantly better than the null model.
